#### Set Up


In [1]:
# imports and dependencies
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [42]:
# dataframes
df_orig = pd.read_csv('data/dataset_original.csv')
df_upd = pd.read_csv('data/dataset_updated.csv', encoding='latin-1')
df_imp = pd.read_csv('data/dataset_imputed.csv', encoding='latin-1')

#### Missing Values


In [3]:
# empty cells in original df
empty_cells_per_column_orig = df_orig.isna().sum()
empty_cells_per_column_orig = empty_cells_per_column_orig.loc[empty_cells_per_column_orig > 0]
empty_cells_per_column_orig

Pos            98
SoT%           23
G/Sh           23
Av Dist        24
npxG/Sh        24
Succ%          31
Compl          10
On-Off          2
On-Off.1        2
2CrdY           1
Fls Comm        1
Fls Drawn       1
PKwon           1
PKcon           1
OG              1
Recov           1
Aerials Won     1
Won%            5
dtype: int64

In [4]:
# empty cells in updated df
empty_cells_per_column_upd = df_upd.isna().sum()
empty_cells_per_column_upd = empty_cells_per_column_upd.loc[empty_cells_per_column_upd > 0]
empty_cells_per_column_upd

Matches Played    292
Av Dist             1
Succ%              31
Compl              10
On-Off              2
On-Off.1            2
2CrdY               1
Fls Comm            1
Fls Drawn           1
PKwon               1
PKcon               1
OG                  1
Recov               1
Aerials Won         1
Won%                5
dtype: int64

#### Pos: Specific Positions


In [5]:
count = 0
for entry in df_orig['Pos']:
    if ',' in str(entry):
        count += 1
count

267

In [6]:
for i in df_orig['Pos'].unique():
    if "," in str(i):
        count = df_orig['Pos'].value_counts()[i]
        print(f"{i} - Count: {count}")

DF,FW - Count: 3
MF,FW - Count: 79
FW,DF - Count: 8
DF,MF - Count: 51
FW,MF - Count: 108
MF,DF - Count: 18


FW,DF (8)

DF,FW (3)

---

MF,FW (79)

FW,MF (108)

---

DF,MF (51)

MF,DF (18)


98 Missing Values

267 Multiple Positions


#### SOT%: Percentage of Shots on Target


23 Missing Values

Divide by Zero


#### G/SH: Goals per Shot


23 Missing Values

Divide by Zero


#### Av Dist: Average Shot Distance


24 Missing Values

23 Divide by Zero

1 Impute


#### NPXG/SH: Non Penalty Expected Goals per Shot


24 Missing Values

All Missing Values had an XG of Zero

NOTE: Issue with this column


#### Succ%: Percentage of Attempted Dribbles Successful


In [7]:
# is Succ% = SuccDribs/Att ?
result = (df_orig['SuccDribs'] / df_orig['Att'])*100 == df_orig['Succ%']
count = sum(result)
count
# ISSUE?

371

**Impute for now**


#### Impute all the following


- On-Off
- On-Off.1
- 2CrdY
- Fls Comm
- Fls Drawn
- PKwon
- PKcon
- OG
- Recov
- Aerials Won
- Won%


#### Imputation


In [8]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

for col in df_upd:
    if df_upd[col].dtypes not in numerics:
        print(col)


Season
Name
Gen Pos
Club
League
Player
Nation
Pos


- **Season:** Integer Encoding
- **Name:** Remove
- **Gen Pos:** One Hot Encoding
- **Club:** One Hot Encoding
- **League:** Remove
- **Player:** Remove
- **Nation:** One Hot Encoding
- **Pos:** One Hot Encoding


In [9]:
df_imp.head()

,Season,Name,Weekly Gross,Annual Gross,Total Gross,Weekly Net,Annual Net,Ln Annual Net,Gen Pos,Club,...,On-Off.1,2CrdY,Fls Comm,Fls Drawn,PKwon,PKcon,OG,Recov,Aerials Won,Won%
0,18/19,Cristián Zapata,50991,2651554,2647903,27537,1431907,14.174518,D,AC Milan,...,0.00,0.0,13.0,8.0,0.0,0.0,0.0,124.0,27.0,75.0
1,19/20,Cristián Zapata,35960,1869902,1869902,19438,1010758,13.826211,D,Genoa,...,0.22,0.0,9.0,7.0,0.0,0.0,0.0,99.0,22.0,66.7
2,20/21,Cristián Zapata,35960,1869902,1869902,19438,1010758,13.826211,D,Genoa,...,-0.29,0.0,20.0,15.0,0.0,1.0,1.0,158.0,59.0,72.8
3,18/19,Duván Zapata,44998,2339904,2336682,24297,1263447,14.049354,F,Atalanta,...,0.03,0.0,31.0,57.0,2.0,0.0,0.0,105.0,62.0,42.5
4,19/20,Duván Zapata,53939,2804852,2804852,29156,1516136,14.231676,F,Atalanta,...,-0.08,0.0,63.0,87.0,0.0,0.0,0.0,135.0,45.0,37.2


In [10]:
# encoders
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder()

# imputer 
imputer = KNNImputer(n_neighbors=10)

In [11]:
# integer encode season
column_to_encode = df_imp["Season"]
encoded_column = label_encoder.fit_transform(column_to_encode)
df_imp["Season"] = encoded_column


In [12]:
# drop player names 
df_imp = df_imp.drop('Name', axis=1)
df_imp = df_imp.drop('Player', axis=1)

In [13]:
# general positions
encoded_columns = pd.get_dummies(df_imp["Gen Pos"], prefix="Gen Pos", dtype = int)
df_imp = pd.concat([df_imp.drop("Gen Pos", axis=1), encoded_columns], axis=1)

In [14]:
# clubs
encoded_columns = pd.get_dummies(df_imp["Club"], prefix="Club", dtype = int )
df_imp = pd.concat([df_imp.drop("Club", axis=1), encoded_columns], axis=1)

In [15]:
# drop league 
df_imp = df_imp.drop('League', axis=1)

In [16]:
# nations
encoded_columns = pd.get_dummies(df_imp["Nation"], prefix="Nation", dtype = int )
df_imp = pd.concat([df_imp.drop("Nation", axis=1), encoded_columns], axis=1)

In [17]:
# positions
encoded_columns = pd.get_dummies(df_imp["Pos"], prefix="Pos", dtype = int)
df_imp = pd.concat([df_imp.drop("Pos", axis=1), encoded_columns], axis=1)

In [18]:
df_imp.replace('', np.nan, inplace=True)

In [19]:
# impute
imputed_data = imputer.fit_transform(df_imp)
df_imputed = pd.DataFrame(imputed_data, columns=df_imp.columns)

In [20]:
df_imputed.head()

,Season,Weekly Gross,Annual Gross,Total Gross,Weekly Net,Annual Net,Ln Annual Net,Transfer In,Age,AgeSQ,...,Pos_LB,Pos_LM,Pos_LW,Pos_MF,"Pos_MF,DF",Pos_MT,Pos_RB,Pos_RM,Pos_RW,Pos_SS
0,0.0,50991.0,2651554.0,2647903.0,27537.0,1431907.0,14.174518,0.0,30.0,900.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,35960.0,1869902.0,1869902.0,19438.0,1010758.0,13.826211,1.0,31.0,961.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,35960.0,1869902.0,1869902.0,19438.0,1010758.0,13.826211,0.0,32.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,44998.0,2339904.0,2336682.0,24297.0,1263447.0,14.049354,1.0,26.0,676.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,53939.0,2804852.0,2804852.0,29156.0,1516136.0,14.231676,0.0,27.0,729.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# empty cells in imputed df
empty_cells_per_column_imputed = df_imputed.isna().sum()
empty_cells_per_column_imputed = empty_cells_per_column_imputed.loc[empty_cells_per_column_imputed > 0]
empty_cells_per_column_imputed

Series([], dtype: int64)

In [22]:
df_imputed

,Season,Weekly Gross,Annual Gross,Total Gross,Weekly Net,Annual Net,Ln Annual Net,Transfer In,Age,AgeSQ,...,Pos_LB,Pos_LM,Pos_LW,Pos_MF,"Pos_MF,DF",Pos_MT,Pos_RB,Pos_RM,Pos_RW,Pos_SS
0,0.0,50991.0,2651554.0,2647903.0,27537.0,1431907.0,14.174518,0.0,30.0,900.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,35960.0,1869902.0,1869902.0,19438.0,1010758.0,13.826211,1.0,31.0,961.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,35960.0,1869902.0,1869902.0,19438.0,1010758.0,13.826211,0.0,32.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,44998.0,2339904.0,2336682.0,24297.0,1263447.0,14.049354,1.0,26.0,676.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,53939.0,2804852.0,2804852.0,29156.0,1516136.0,14.231676,0.0,27.0,729.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,4.0,30960.0,1609936.0,6439744.0,24188.0,1257763.0,14.044845,0.0,27.0,729.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1622,3.0,31313.0,1628267.0,8141336.0,24463.0,1272084.0,14.056167,1.0,26.0,676.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1623,1.0,145296.0,7555413.0,7555413.0,113386.0,5896086.0,15.589799,1.0,27.0,729.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1624,2.0,145296.0,7555413.0,7555413.0,113386.0,5896086.0,15.589799,0.0,28.0,784.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df_imputed.to_csv('imputed_df.csv', index=False)


In [31]:
for index, row in df_imputed.iterrows():
    count_ones = sum(row[["Gen Pos_D", "Gen Pos_M", "Gen Pos_F"]] == 1)
    count_zeros = sum(row[["Gen Pos_D", "Gen Pos_M", "Gen Pos_F"]] == 0)

    if count_ones != 1 or count_zeros != 2:
        print(row["Weekly Gross"])

In [43]:
df_upd["Nation"].unique()

array(['Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Austria',
       'Belgium', 'Bosnia and Herzegovina', 'Brazil', 'Burkina Faso',
       'Cameroon', 'Chile', 'Colombia', 'Costa Rica', "Côte d'Ivoire",
       'Croatia', 'Czechia', 'Denmark', 'Ecuador', 'England',
       'Equatorial Guinea', 'Estonia', 'Finland', 'France', 'The Gambia',
       'Germany', 'Ghana', 'Greece', 'Guinea', 'Guinea Bissau', 'Hungary',
       'Iraq', 'Israel', 'Italy', 'Japan', 'Kosovo', 'Martinique',
       'Mexico', 'Moldova', 'Montenegro', 'Morocco', 'Netherlands',
       'New Zealand', 'Nigeria', 'North Korea', 'North Macedonia',
       'Norway', 'Paraguay', 'Peru', 'Poland', 'Portugal', 'Romania',
       'Russia', 'Scotland', 'Senegal', 'Serbia', 'Slovakia', 'Slovenia',
       'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
       'United States', 'Uruguay', 'Uzbekistan', 'Venezuela', 'Wales'],
      dtype=object)